In [1]:
#Brker CNN arkitektur kombinert med PyTorch 


In [2]:
import numpy as np
import matplotlib.pyplot as plt
# import pandas as pd

#other libraries
# from tqdm import tqdm
# import time
# import random
import os
import sys
from pathlib import Path
import h5py

#torch specific
import torch
import torchvision as torchv
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch import Tensor
from torch.utils import data

from sklearn.model_selection import train_test_split
import wandb

from sklearn.metrics import confusion_matrix
import pandas as pd
import seaborn as sn

c:\Users\student\Anaconda3\envs\DAT191-gpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
module_path = str(Path.cwd().parents[0].parents[0] / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

from dataloader import *
# from plotCreator import *

data_path0 = str(Path.cwd().parents[0].parents[0] / "data" / "bh" / "BH_n4_M10_res50_15000_events.h5")
data_path1 = str(Path.cwd().parents[0].parents[0] / "data" / "sph" / "PP13-Sphaleron-THR9-FRZ15-NB0-NSUBPALL_res50_15000_events.h5")

In [4]:
bhArray = dataToArray(data_path0)
sphArray = dataToArray(data_path1)

In [5]:
print(bhArray.shape)
print(sphArray.shape)

(15000, 50, 50, 3)
(15000, 50, 50, 3)


In [6]:
dataArray = np.concatenate((bhArray,sphArray),axis=0)

In [7]:
np.shape(dataArray)

(30000, 50, 50, 3)

In [8]:
labelsArray = np.concatenate((np.zeros(np.shape(bhArray)[0]),np.ones(np.shape(sphArray)[0])),axis=0)

In [9]:
np.shape(labelsArray)

(30000,)

In [10]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [11]:
trainData, testData, trainLabels, testLabels = train_test_split(dataArray, labelsArray, random_state=42)

In [12]:
trainData = torch.from_numpy(trainData)
testData = torch.from_numpy(testData)
trainLabels = torch.from_numpy(trainLabels)
testLabels = torch.from_numpy(testLabels)

In [13]:
train = torch.utils.data.TensorDataset(trainData, trainLabels)
test = torch.utils.data.TensorDataset(testData, testLabels)

In [14]:
trainLoader = DataLoader(train, shuffle=True, batch_size=50)
testLoader = DataLoader(test, shuffle=True, batch_size=50)

In [15]:
class LinearModel(nn.Module):
    def __init__(self, resolution, num_classes, stride=1):

        super(LinearModel, self).__init__()
        self.fc1 = nn.Linear(3, 1000)
        self.fc2 = nn.Linear(1000, 100)
        self.fc3 = nn.Linear(100, 2)
        


    def forward(self, x:Tensor):
        out = F.relu(self.fc1(x))
        out = F.relu(self.fc2(x))
        out = self.fc3(x)
        return out

In [16]:
class ConvModel(nn.Module):
    def __init__(self):

        super(ConvModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=0)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=64, kernel_size=3, padding=0)

        self.fc1 = nn.Linear(11*11*64, 128)
        self.fc2 = nn.Linear(128,2)


    def forward(self, x:Tensor):
        x = self.conv1(x)
        x = F.relu(x) #to activate function above

        x = F.max_pool2d(x,2)

        x = self.conv2(x)
        x = F.relu(x)

        x = F.max_pool2d(x,2)

        x = torch.flatten(x, 1)

        x = self.fc1(x)
        x = F.relu(x)

        x = self.fc2(x)
 
        return x

In [17]:
convModel = ConvModel().to(device)

In [18]:
trainData.shape

torch.Size([22500, 50, 50, 3])

In [19]:
trainFlipped = trainData.permute(0,3,1,2)

In [20]:
trainFlipped.shape

torch.Size([22500, 3, 50, 50])

In [22]:
trainLabels = trainLabels.type(torch.LongTensor)

In [ ]:
def train():

    
    convModel.train()
    for epoch in range(wandb.config['epochs']):

        trainRunningLoss = 0.0
        correct = 0
        y_pred = []
        y_true = []
        for i, data in enumerate(trainLoader):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            output = convModel(inputs.permute(0,3,1,2))
            output1 = (torch.max(torch.exp(output.to(device)), 1)[1])

            y_pred.extend(output1) # Save Prediction
            
            y_true.extend(labels) # Save Truth

            loss = criterion(output, labels.type(torch.LongTensor).to(device))
            loss.backward()
            optimizer.step()
            
            trainRunningLoss += loss.item()
        
        correct = (torch.FloatTensor(y_pred) == torch.FloatTensor(y_true)).sum()
        trainAccuracy = correct / len(y_true)
        convModel.eval()

        testRunningLoss = 0.0
        y_pred = []
        y_true = []
        for j, data in enumerate(testLoader):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            output = convModel(inputs.permute(0,3,1,2))# Feed Network

            output1 = (torch.max(torch.exp(output.to(device)), 1)[1])
            y_pred.extend(output1) # Save Prediction
            
            y_true.extend(labels) # Save Truth
            loss = criterion(output, labels.type(torch.LongTensor).to(device))
            testRunningLoss += loss.item()

        convModel.train()

        correct = (torch.FloatTensor(y_pred) == torch.FloatTensor(y_true)).sum()

        testAccuracy = correct / len(y_true)
        epochLoss = trainRunningLoss/len(bhArray)
        wandb.log({"Train epoch_loss":epochLoss, "Test epoch_loss": testRunningLoss, "Train accuracy": trainAccuracy,"Test accuracy": testAccuracy})

In [ ]:
#Weights and biases set up
wandb.init(
    # set the wandb project where this run will be logged
    name="convmodel",
    project="PyTorch",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "epochs": 50,
    }
)

In [38]:

num_epochs = wandb.config['epochs']
learning_rate = wandb.config['learning_rate']
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(convModel.parameters(), lr=learning_rate)

wandb.watch(convModel, criterion, log='all', log_freq=100)

train()

wandb.finish()

Test accuracy,▁▇▅▄▇▆▁▂▅▅▃▄▅▆▆███████▇█████████████████
Test epoch_loss,▁▁▁▁▂▃▃▃▄▄▄▄▄▅▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇███████████
Train accuracy,▁▂▄▄▆▆▇▇▇▇▇▇▇███████████████████████████
Train epoch_loss,█▇▆▅▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test accuracy,0.82093
Test epoch_loss,223.14542
Train accuracy,1.0
Train epoch_loss,0.0


# Hyperparamater testing